In [21]:
#pip install opencv-python imutils pytesseract numpy

In [2]:
import cv2
import imutils
import pytesseract
import numpy as np
from datetime import datetime
from tkinter import Tk
from tkinter.filedialog import askopenfilename

In [3]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [4]:
with open("blacklist.csv", "r") as f:
    blacklist = [line.strip() for line in f.readlines()]

In [5]:
image = cv2.imread("bike.jpg")
image = imutils.resize(image, width=600)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [6]:
gray = cv2.bilateralFilter(gray, 11, 17, 17)
edged = cv2.Canny(gray, 30, 200)

In [7]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]

In [8]:

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
plate_cascade = cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')

if plate_cascade.empty():
    raise IOError("Cascade file not loaded!")

    blacklist = [line.strip() for line in f.readlines()]

Tk().withdraw()
file_path = askopenfilename(title="Select Car/Bike Image", filetypes=[("Image files", "*.jpg *.png *.jpeg")])

if not file_path:
    print("No file selected!")
    exit()

image = cv2.imread(file_path)
if image is None:
    print("Failed to load image!")
    exit()

image = cv2.imread(file_path)
image = cv2.resize(image, (1000, 700))  


plates = plate_cascade.detectMultiScale(gray, 1.1, 10)

if len(plates) == 0:
    print("No license plate detected.")
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    for (x, y, w, h) in plates:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        plate_img = gray[y:y+h, x:x+w]

 
        plate_img = cv2.bilateralFilter(plate_img, 11, 17, 17)
        _, plate_img = cv2.threshold(plate_img, 150, 255, cv2.THRESH_BINARY)

 
        text = pytesseract.image_to_string(
            plate_img,
            config="--psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
        )
        plate_text = "".join(filter(str.isalnum, text))
        print("Detected Plate:", plate_text)

        if plate_text in blacklist:
            status = "ALERT: STOLEN VEHICLE DETECTED"
            color = (0, 0, 255)
        else:
            status = "Vehicle is SAFE"
            color = (0, 255, 0)

        cv2.putText(image, status, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        cv2.putText(image, plate_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # Log result
        with open("log.txt", "a", encoding="utf-8") as log:
            log.write(f"{plate_text} | {status} | {datetime.now()}\n")

No file selected!
Failed to load image!


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
plateCnt = None

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
    if len(approx) == 4:
        plateCnt = approx
        break


if plateCnt is None:
    print("License plate not detected")
else:

    cv2.drawContours(image, [plateCnt], -1, (0, 255, 0), 3)

    mask = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(mask, [plateCnt], 0, 255, -1)
    masked = cv2.bitwise_and(gray, gray, mask=mask)

    x, y, w, h = cv2.boundingRect(plateCnt)
    plate_img = masked[y:y+h, x:x+w]


    text = pytesseract.image_to_string(
        plate_img,
        config="--psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    )

    plate_text = "".join(filter(str.isalnum, text))
    print("Detected Plate:", plate_text)


    if plate_text in blacklist:
        status = "ALERT: STOLEN VEHICLE DETECTED"
        color = (0, 0, 255)
    else:
        status = "VEHICLE IS SAFE"
        color = (0, 255, 0)

    cv2.putText(image, status, (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)


    with open("log.txt", "a", encoding="utf-8") as log:
        log.write(f"{plate_text} | {status} | {datetime.now()}\n")

    cv2.imshow("Detected License Plate", plate_img)


cv2.imshow("Stolen Vehicle Detection System", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)  

while True:
    ret, frame = cap.read()
    if not ret:
        break

   
    cv2.imshow("Webcam", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
cv2.putText(frame, "ALERT", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
cv2.imshow("Webcam", frame) 

cap.release()
cv2.destroyAllWindows()